## Kafka Producer

In [1]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewPartitions, NewTopic
from faker import Faker
from time import sleep

In [2]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
kafka_host = os.getenv('KAFKA_HOST')

In [4]:
print(kafka_host)

dataeng-kafka


In [5]:
producer = KafkaProducer(bootstrap_servers=f'{kafka_host}:9092')

In [6]:
_instance = Faker()
global faker
# Inisialisasi Faker
faker = Faker()

In [7]:
# Membuat instance KafkaAdminClient
admin_client = KafkaAdminClient(bootstrap_servers=f'{kafka_host}:9092')

# Mendefinisikan topik yang ingin dibuat
kafka_topic_partition = "Data_Transaksi_Keuangan_E-commerce"
partitions = 5  # Jumlah partisi
replication_factor = 1  # Faktor replikasi, sesuaikan dengan kebutuhan

# Membuat objek NewTopic
new_topic = NewTopic(name=kafka_topic_partition, num_partitions=partitions, replication_factor=replication_factor)

# Membuat topik
try:
    admin_client.create_topics([new_topic])
    print(f"Topik '{kafka_topic_partition}' berhasil dibuat dengan {partitions} partisi.")
except Exception as e:
    print(f"Terjadi kesalahan saat membuat topik '{kafka_topic_partition}': {e}")

Topik 'Data_Transaksi_Keuangan_E-commerce' berhasil dibuat dengan 5 partisi.


In [8]:
# Di sini diasumsikan Anda sudah memiliki producer Kafka dan topik Kafka
# producer = <kafka_producer_instance>
# kafka_topic_partition = "<kafka_topic_name>"

kafka_topic_partition = "Data_Transaksi_Keuangan_E-commerce"

class DataGenerator(object):
    @staticmethod
    def get_data():
        # Generate individual data items
        transaction_id = uuid.uuid4().__str__()  # transaction_id
        timestamp = faker.date_time_this_year().isoformat()  # timestamp
        user_id = faker.uuid4()  # user_id
        merchant_id = faker.uuid4()  # merchant_id
        amount = round(faker.random_number(digits=5) + faker.random.random(), 2)  # amount
        age = faker.random_int(min=19, max=65)  # age
        gender = faker.random_element(elements=('Male', 'Female'))  # gender
        phone_number = faker.phone_number()  # phone_number
        email = faker.email()  # email
        currency = faker.currency_code()  # currency
        location = faker.city()  # location
        country = faker.country()  # country
        payment_method = faker.random_element(elements=("Credit Card", "Debit Card", "PayPal", "Local Bank Transfer", "Foreign Bank Transfer"))  # payment_method
        product_id = faker.uuid4()  # product_id
        product_category = faker.random_element(elements=( "Electronics and Gadgets",
                                                            "Fashion and Accessories",
                                                            "Health and Beauty",
                                                            "Home and Furniture",
                                                            "Food and Beverages",
                                                            "Hobbies and Collectibles",
                                                            "Sports and Outdoors",
                                                            "Automotive",
                                                            "Baby and Kids Products",
                                                            "Office and School Supplies",
                                                            "Kitchen and Cooking Supplies",
                                                            "Digital Products and Services",
                                                            "Pet Supplies",
                                                            "Real Estate and Property",
                                                            "Services and Consultancy" ))  # product_category
        quantity = faker.random_int(min=1, max=50)  # quantity
        discount = round(faker.random_number(digits=2), 2)  # discount
        tax = round(faker.random_number(digits=2), 2)  # tax
        shipping_cost = round(faker.random_number(digits=2), 2)  # shipping_cost

        # Hitung total_cost di sini
        total_cost = round(amount - discount + tax + shipping_cost, 2)  # Hitung total_cost
        
        return [
            transaction_id, timestamp, user_id, merchant_id, amount, age, gender, 
            phone_number, email, currency, location, country, payment_method, product_id, 
            product_category, quantity, discount, tax, shipping_cost, total_cost  # total_cost sudah dihitung dan ditambahkan di sini
        ]

# Loop untuk mengirim data ke Kafka
for x in range(1, 500):
    columns = [
        "transaction_id", "timestamp", "user_id", "merchant_id", "amount", "age", "gender", 
        "phone_number", "email","currency", "location", "country", "payment_method", "product_id", 
        "product_category", "quantity", "discount", "tax", "shipping_cost", "total_cost"
    ]
    data_list = DataGenerator.get_data()
    json_data = dict(zip(columns, data_list))
    _payload = json.dumps(json_data).encode("utf-8")
    
    # Menggunakan transaction_id sebagai kunci untuk menentukan partisi
    key = json_data['transaction_id'].encode("utf-8")  # Kunci sebagai bytes
    
    # Mengirim data ke Kafka (pastikan `producer` dan `kafka_topic_partition` sudah didefinisikan)
    response = producer.send(topic=kafka_topic_partition, value=_payload, key=key)
    print(json_data['transaction_id'], response.get())
    
    # Untuk testing, kita hanya akan print data
    # print(json_data)
    
    sleep(5)  # Jeda 5 detik sebelum mengirim data berikutnya


7cd79e36-d4ca-4e80-9b09-2db33c6f2d6c RecordMetadata(topic='Data_Transaksi_Keuangan_E-commerce', partition=4, topic_partition=TopicPartition(topic='Data_Transaksi_Keuangan_E-commerce', partition=4), offset=0, timestamp=1730881650404, log_start_offset=0, checksum=None, serialized_key_size=36, serialized_value_size=643, serialized_header_size=-1)
5887c84e-bde2-438b-bb19-b6858cc61a2b RecordMetadata(topic='Data_Transaksi_Keuangan_E-commerce', partition=1, topic_partition=TopicPartition(topic='Data_Transaksi_Keuangan_E-commerce', partition=1), offset=0, timestamp=1730881655410, log_start_offset=0, checksum=None, serialized_key_size=36, serialized_value_size=592, serialized_header_size=-1)
089d427e-dae4-4b8d-9dd2-f2176b862644 RecordMetadata(topic='Data_Transaksi_Keuangan_E-commerce', partition=1, topic_partition=TopicPartition(topic='Data_Transaksi_Keuangan_E-commerce', partition=1), offset=1, timestamp=1730881660418, log_start_offset=0, checksum=None, serialized_key_size=36, serialized_value

KeyboardInterrupt: 